In [ ]:
from nltk.corpus import sentiwordnet as swn
import nltk
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize, pos_tag
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('sentiwordnet')

In [ ]:
# 데이터 가져오기
df = pd.read_csv('cooking.csv', encoding = 'utf-8')
# 빈칸 있는 행 제거
df.dropna(inplace = True)
# comment 에 있는 문자열 다 소문자로 변경
df['comment'] = df['comment'].str.lower()
# comment 에 있는 글들 리스트에 저장
sentence = list(df['comment'])
# 빈 데이터프레임 생성
result = pd.DataFrame(columns = ["comment", "n_p-n", "v_p-n", "r_p-n", "sa_p-n"])
# 불용어 사전
stop_words = set(stopwords.words('english'))
# 문자열 하나씩 가져와서 점수 생성하기
for i in range(len(sentence)) :

    comment = sentence[i]
    words = []
    # 토큰화
    sent = pos_tag(word_tokenize(comment))
    # 명사, 부사, 형용사, 동사 중 하나이고, 불용어가 아니면서 2글자 이상의 단어들만 저장
    for s in sent :
        if s[1] in ('NN', 'VB', 'VBG', 'VERB', 'ADV', 'JJ', 'ADJ') and s[0] not in stop_words :
            if len(s[0]) > 2 :
                words.append((s[0]))
    # 한 문장에 겹치는 단어 없게끔 저장
    words = list(set(words))

#     words = list(set(comment.split(' ')))
    # 명사
    n_pos_avg = []
    n_neg_avg = []
    # 동사
    v_pos_avg = []
    v_neg_avg = []
    # 부사
    r_pos_avg = []
    r_neg_avg = []
    # 형용사
    sa_pos_avg = []
    sa_neg_avg = []
    
    # 처리된 단어가 있는 문장만 실행
    try :
        for k in words :
            word_kind = list(swn.senti_synsets(k))

            # 명사
            n_pos = []
            n_neg = []
            # 동사
            v_pos = []
            v_neg = []
            # 부사
            r_pos = []
            r_neg = []
            # 형용사
            sa_pos = []
            sa_neg = []
            
            # 점수 가져오기
            for word in word_kind:

                synset = swn.senti_synset(str(word)[1:-1].split(':')[0])
                temp= str(synset)[1:-1].split()

                if temp[1].split('=')[1] != '0.0' or temp[2].split('=')[1] != '0.0' :

                    if temp[0][:-1].split('.')[1] == 'n' :
                        n_pos.append(float(temp[1].split('=')[1]))
                        n_neg.append(float(temp[2].split('=')[1]))
                    elif temp[0][:-1].split('.')[1] == 'v' :
                        v_pos.append(float(temp[1].split('=')[1]))
                        v_neg.append(float(temp[2].split('=')[1]))
                    elif temp[0][:-1].split('.')[1] == 'r' :
                        r_pos.append(float(temp[1].split('=')[1]))
                        r_neg.append(float(temp[2].split('=')[1]))
                    else :
                        sa_pos.append(float(temp[1].split('=')[1]))
                        sa_neg.append(float(temp[2].split('=')[1]))

            n_sum_p = 0
            n_sum_n = 0
            v_sum_p = 0
            v_sum_n = 0
            r_sum_p = 0
            r_sum_n = 0 
            sa_sum_p = 0
            sa_sum_n = 0
            
            # 단어별로 품사구분하여 점수의 평균값 저장
            if len(n_pos) != 0 and len(n_neg) != 0 :

                for i in n_pos :
                    n_sum_p += i
                for i in n_neg :
                    n_sum_n += i

                n_pos_avg.append(round((n_sum_p / len(n_pos)), 3))
                n_neg_avg.append(round((n_sum_n / len(n_neg)), 3))
            elif len(n_pos) == 0 and len(n_neg) == 0:
                n_pos_avg.append(0.000)
                n_neg_avg.append(0.000)
            if len(v_pos) != 0 and len(v_neg) != 0 :

                for i in v_pos :
                    v_sum_p += i
                for i in v_neg :
                    v_sum_n += i

                v_pos_avg.append(round((v_sum_p / len(v_pos)), 3))
                v_neg_avg.append(round((v_sum_n / len(v_neg)), 3))
            elif len(v_pos) == 0 and len(v_neg) == 0:
                v_pos_avg.append(0.000)
                v_neg_avg.append(0.000)
            if len(r_pos) != 0 and len(r_neg) != 0 :

                for i in r_pos :
                    r_sum_p += i
                for i in r_neg :
                    r_sum_n += i

                r_pos_avg.append(round((r_sum_p / len(r_pos)), 3))
                r_neg_avg.append(round((r_sum_n / len(r_neg)), 3))
            elif len(r_pos) == 0 and len(r_neg) == 0:
                r_pos_avg.append(0.000)
                r_neg_avg.append(0.000)
            if len(sa_pos) != 0 and len(sa_neg) != 0 :

                for i in sa_pos :
                    sa_sum_p += i
                for i in sa_neg :
                    sa_sum_n += i

                sa_pos_avg.append(round((sa_sum_p / len(sa_pos)), 3))
                sa_neg_avg.append(round((sa_sum_n / len(sa_neg)), 3))
            elif len(sa_pos) == 0 and len(sa_neg) == 0:
                sa_pos_avg.append(0.000)
                sa_neg_avg.append(0.000)

        result=result.append({'comment' : comment, 
                              'n_p-n' :  max(n_pos_avg) - max(n_neg_avg), 
                              'v_p-n' : max(v_pos_avg) - max(v_neg_avg),
                              'r_p-n' : max(r_pos_avg) - max(r_neg_avg),
                              'sa_p-n' : max(sa_pos_avg) - max(sa_neg_avg),
                             }, ignore_index = True)
    except :
        continue

result.to_csv('anal_cooking.csv', encoding = 'utf-8', index = False)